# Imports

In [6]:
import sys, os, importlib, pickle

path2add = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir, 'sql_generator')))

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

#print(path2add)

#import index
#import llm
#import db
#import util

from index import Index
from llm import LLM, PromptGenerator


/Users/vrabinin/Documents/Github/llm-zoomcamp-capstone/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def reimport_all():
    import index
    import llm
    import db
    import util
    importlib.reload(index)
    importlib.reload(llm)
    importlib.reload(db)
    importlib.reload(util)

# Index Data

## Extract DB Metadata

In [10]:
import db
importlib.reload(db)
from db import DBConnection
db_conn = DBConnection(db_type='postgresql',
                      db_name='dvdrental',
                      db_host='localhost',
                      db_port=os.environ['POSTGRES_L_PORT'], 
                      db_user=os.environ['POSTGRES_USER'] ,
                      db_password=os.environ['POSTGRES_PASSWORD'])

db_list = db_conn.get_database_list()
print(db_list)

md = db_conn.get_metadata(exclude_system_tables=True)
md.df_table_columns.head(20)


['adventureworks', 'dvdrental']


,table_schema,table_name,table_type,column_name,data_type,is_primary_key,is_nullable
0,public,actor,table,"[actor_id, first_name, last_name, last_update]","[INTEGER, VARCHAR(45), VARCHAR(45), TIMESTAMP]","[True, False, False, False]","[False, False, False, False]"
1,public,store,table,"[store_id, manager_staff_id, address_id, last_...","[INTEGER, SMALLINT, SMALLINT, TIMESTAMP]","[True, False, False, False]","[False, False, False, False]"
2,public,address,table,"[address_id, address, address2, district, city...","[INTEGER, VARCHAR(50), VARCHAR(50), VARCHAR(20...","[True, False, False, False, False, False, Fals...","[False, False, True, False, False, True, False..."
3,public,city,table,"[city_id, city, country_id, last_update]","[INTEGER, VARCHAR(50), SMALLINT, TIMESTAMP]","[True, False, False, False]","[False, False, False, False]"
4,public,country,table,"[country_id, country, last_update]","[INTEGER, VARCHAR(50), TIMESTAMP]","[True, False, False]","[False, False, False]"
5,public,staff,table,"[staff_id, first_name, last_name, address_id, ...","[INTEGER, VARCHAR(45), VARCHAR(45), SMALLINT, ...","[True, False, False, False, False, False, Fals...","[False, False, False, False, True, False, Fals..."
6,public,category,table,"[category_id, name, last_update]","[INTEGER, VARCHAR(25), TIMESTAMP]","[True, False, False]","[False, False, False]"
7,public,customer,table,"[customer_id, store_id, first_name, last_name,...","[INTEGER, SMALLINT, VARCHAR(45), VARCHAR(45), ...","[True, False, False, False, False, False, Fals...","[False, False, False, False, True, False, Fals..."
8,public,film_actor,table,"[actor_id, film_id, last_update]","[SMALLINT, SMALLINT, TIMESTAMP]","[True, True, False]","[False, False, False]"
9,public,film,table,"[film_id, title, description, release_year, la...","[INTEGER, VARCHAR(255), TEXT, DOMAIN, SMALLINT...","[True, False, False, False, False, False, Fals...","[False, False, True, True, False, False, False..."


In [15]:
db_conn.set_curr_database('adventureworks')
df_db_metadata = db_conn.get_metadata(refresh=True, exclude_system_tables=True)
df_db_metadata.df_table_columns.head()

,table_schema,table_name,table_type,column_name,data_type,is_primary_key,is_nullable
0,humanresources,department,table,"[departmentid, name, groupname, modifieddate]","[INTEGER, DOMAIN, DOMAIN, TIMESTAMP]","[True, False, False, False]","[False, False, False, False]"
1,humanresources,shift,table,"[shiftid, name, starttime, endtime, modifieddate]","[INTEGER, DOMAIN, TIME, TIME, TIMESTAMP]","[True, False, False, False, False]","[False, False, False, False, False]"
2,humanresources,employeepayhistory,table,"[businessentityid, ratechangedate, rate, payfr...","[INTEGER, TIMESTAMP, NUMERIC, SMALLINT, TIMEST...","[True, True, False, False, False]","[False, False, False, False, False]"
3,humanresources,employee,table,"[businessentityid, nationalidnumber, loginid, ...","[INTEGER, VARCHAR(15), VARCHAR(256), VARCHAR(5...","[True, False, False, False, False, False, Fals...","[False, False, False, False, False, False, Fal..."
4,person,person,table,"[businessentityid, persontype, namestyle, titl...","[INTEGER, CHAR(2), DOMAIN, VARCHAR(8), DOMAIN,...","[True, False, False, False, False, False, Fals...","[False, False, False, True, False, True, False..."


In [5]:
db_metadata_all = {}

for db_name in db_list:
    db_conn.set_curr_database(db_name)
    df_db_metadata = db_conn.get_metadata()
    db_metadata_all[db_name] = df_db_metadata

print(db_metadata_all)

{'postgres': <db.DBMetadata object at 0x331599110>, 'sql_generator': <db.DBMetadata object at 0x33167f0d0>, 'dagster': <db.DBMetadata object at 0x3316d4590>, 'adventureworks': <db.DBMetadata object at 0x3317efb10>, 'dvdrental': <db.DBMetadata object at 0x330f1b150>}


## Create Index in Fuzzy Search

In [4]:
import search
import db
importlib.reload(search)
importlib.reload(db)
from search import SearchFactory, SearchTypes
db_conn1 = db.DBConnection(db_type='postgresql',
                      db_name='sql_generator',
                      db_host='localhost',
                      db_port=os.environ['POSTGRES_L_PORT'], 
                      db_user=os.environ['POSTGRES_USER'] ,
                      db_password=os.environ['POSTGRES_PASSWORD'],
                      read_only=False)

my_search = SearchFactory.get_search_provider(SearchTypes.FUZZY_SEARCH, db_conn=db_conn1)

for key, value in db_metadata_all.items():
    print(key)
    my_search.create_index(key, value)
    


ModuleNotFoundError: No module named 'search'

# Search in Database Metadata

In [8]:
# metadata = pickle.loads(db_conn1.load_file("adventureworks_table_columns.pkl"))
metadata = pickle.loads(db_conn1.load_file("dvdrental_table_columns.pkl"))
metadata.head()

NameError: name 'db_conn1' is not defined

In [9]:
query = "What is the total revenue for top 5 cutomers?"

test_result = my_search.search_by_query(db_name='dvdrental', query=query, similarity_threshold=80, max_synonyms=5)


print(test_result)

NameError: name 'my_search' is not defined

In [10]:
print( sys.path)


['/Users/vrabinin/.pyenv/versions/3.11.10/lib/python311.zip', '/Users/vrabinin/.pyenv/versions/3.11.10/lib/python3.11', '/Users/vrabinin/.pyenv/versions/3.11.10/lib/python3.11/lib-dynload', '', '/Users/vrabinin/Documents/Github/llm-zoomcamp-capstone/.venv/lib/python3.11/site-packages', '/Users/vrabinin/Documents/Github/llm-zoomcamp-capstone/sql_generator']


In [11]:
import db
importlib.reload(db)
from db import DBConnection

db_meta = DBConnection(db_type='postgresql',
                      db_name='dvdrental',
                      db_host='localhost',
                      
                      db_port=os.environ['POSTGRES_L_PORT'], 
                      db_user=os.environ['POSTGRES_USER'] ,
                      db_password=os.environ['POSTGRES_PASSWORD'])



In [12]:
print(db_meta.get_database_list())
db_meta.set_curr_database('dvdrental')

['adventureworks', 'dvdrental']


# RAG Flow

In [13]:
import llm
reimport_all()

from llm import LLM, PromptGenerator
template_path = '../sql_generator/templates'
config_path = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir, 'sql_generator/config/llm.yaml')))
pg = PromptGenerator(template_path)

my_llm = LLM(config_path)
models = my_llm.get_model_list()
print(models)
print(template_path)


['OpenAI: gpt-4o-mini', 'OpenAI: gpt-3.5-turbo', 'TBD: tbd']
../sql_generator/templates


In [14]:
import db
importlib.reload(db)
from db import DBConnection
db_conn = DBConnection(\
        db_type='postgresql',
        db_name = os.environ['POSTGRES_DB'],
        db_host='localhost',
        db_port = os.environ['POSTGRES_L_PORT'], 
        db_user = os.environ['POSTGRES_USER'] ,
        db_password = os.environ['POSTGRES_PASSWORD']
    )

In [28]:
import search
importlib.reload(search)
importlib.reload(db)
from search import SearchFactory, SearchTypes
my_search = SearchFactory.get_search_provider(SearchTypes.FUZZY_SEARCH, db_conn=db_conn)


In [29]:

#query = "What is the total revenue for top 5 cutomers?"

query = "What were the most popular films in July 2005?"
#query = "Are semicolons optional in JavaScript?"


#query = "Show me actor's first_name, last_name that have Nick, Ed and Jennifer as their firstnames"

In [30]:
discovered_tables = my_search.search_by_query(db_name='adventureworks', query=query, similarity_threshold=80, max_synonyms=5)
print(discovered_tables)


Loading metadata for adventureworks
Empty DataFrame
Columns: [table_name, column_name]
Index: []


In [44]:
prompt = pg.get_prompt(template_name='basic_prompt', schema=discovered_tables, instruction=query)
#print(prompt)

Here is a database schema: 
{schema}
  Please write me a syntactically correct SQL statement that answers the following question: {instruction}
  It is not necessary to use all the tables. Use tag <SQL> to wrap the SQL statement.


In [45]:
response = my_llm.prompt(models[0], prompt)
#print(response)

gpt-4o-miniHere is a database schema: 
Empty DataFrame
Columns: [table_schema, table_name, table_type, column_name, data_type, is_primary_key, is_nullable]
Index: []
  Please write me a syntactically correct SQL statement that answers the following question: What were the most popular films in July 2005?
  It is not necessary to use all the tables. Use tag <SQL> to wrap the SQL statement.


In [42]:
import re
response = my_llm.rag_query(query, related_tables, idx_minsearch)
print(response)
sql_pattern = r"<SQL>([\s\S]*?)(?=<\/SQL>)"
sql = str(re.search(sql_pattern, response).group(1)).strip()
print(sql)

NameError: name 'related_tables' is not defined

In [14]:
from openai import OpenAI    
llm = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
response = llm.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
            #{
            #    "role": "system",
            #    "content": "You are a helpful assistant."
            #},
            {"role": "user", "content": query}
        ],
        stream=False
        )

In [15]:
print(response.choices[0].message.content)

Yes, semicolons are technically optional in JavaScript due to a feature called Automatic Semicolon Insertion (ASI). JavaScript interpreters automatically insert semicolons in certain situations where they are omitted, which can help prevent errors in many cases.

However, relying on ASI can lead to unexpected behavior and hard-to-debug issues. Here are some important points to consider:

1. **When ASI Works**: In many cases, ASI will correctly insert semicolons for you. For example:
   ```javascript
   let a = 5
   let b = 10
   console.log(a + b) // ASI inserts semicolons here
   ```

2. **Potential Pitfalls**: There are scenarios where ASI can lead to unexpected results:
   ```javascript
   function example() {
       return
       {
           key: 'value'
       }
   }
   ```
   In this case, the `return` statement will be interpreted as `return;` (returning `undefined`) because of a newline after `return`.

3. **Best Practices**: It is generally recommended to use semicolons consi